---
Cell 0 — Pipeline Switches (0/1)
---
---

In [1]:
# Pipeline mode:
# 0 = CNN on time-series (your current Torch training)
# 1 = Feature-based ML (Logistic Regression with optional feature selection + postprocess)
PIPELINE_MODE = 1

# 0/1 switches
ENABLE_BIPOLAR = 1
ENABLE_OVERLAP_AUGMENT = 1
ENABLE_WITHIN_SUBJECT_STANDARDIZE = 1

ENABLE_FEATURE_EXTRACTION = 0
ENABLE_FEATURE_SELECTION = 0
ENABLE_POSTPROCESSING = 1

# Recommended consistent settings:
# - CNN pipeline: PIPELINE_MODE=0, ENABLE_FEATURE_EXTRACTION=0, ENABLE_FEATURE_SELECTION=0
# - Feature pipeline: PIPELINE_MODE=1, ENABLE_FEATURE_EXTRACTION=1, ENABLE_FEATURE_SELECTION=1

# Feature extraction settings (used when ENABLE_FEATURE_EXTRACTION=1)
FS = 128.0  # sampling rate (Hz). Change if your dataset uses a different FS.

BANDS = {
    "delta": (0.5, 4.0),
    "theta": (4.0, 8.0),
    "alpha": (8.0, 13.0),
    "beta":  (13.0, 30.0),
    "gamma": (30.0, 45.0),
}

# Feature selection settings (used when ENABLE_FEATURE_SELECTION=1)
FEATURE_SELECTION_METHOD = "f_classif"  # "f_classif" or "mutual_info"
FEATURE_K = 200  # number of selected features (top-K)

# Post-processing settings (used when ENABLE_POSTPROCESSING=1)
SMOOTH_WINDOW = 5  # odd integer recommended (e.g., 3,5,7)

# Training settings
N_RUNS = 11
RANDOM_SEED = 42

# Torch CNN settings (PIPELINE_MODE=0)
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
N_EPOCHS = 15

# --------- IMPORTANT: Keep your original cell structure + naming intact ---------
# (As requested, I won't rename or re-section any cells.)

!cp "/kaggle/input/dataset-drowsiness/dataset (1).mat" /kaggle/working/dataset.mat

# Check
!ls -lh /kaggle/working/
print("\n✓ Dataset copied")

import os
os.environ['TORCH_COMPILE_DISABLE'] = '1'
os.environ['TORCHDYNAMO_DISABLE'] = '1'

import torch
torch._dynamo.config.suppress_errors = True

import torch.nn as nn
import torch.optim as optim
import scipy.io as sio
import numpy as np

# --- Post-processing utility (defined early so it can be used during evaluation) ---
def smooth_predictions(pred, window=5):
    pred = np.asarray(pred).astype(int)
    if window is None or window <= 1:
        return pred
    if window % 2 == 0:
        window += 1

    pad = window // 2
    padded = np.pad(pred, (pad, pad), mode="edge")
    out = pred.copy()

    for i in range(len(pred)):
        w = padded[i:i + window]
        out[i] = 1 if np.mean(w) >= 0.5 else 0
    return out

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GroupKFold

# Electrode indices
fp1, fp2, f7 = 0, 1, 2
f3, fz, f4 = 3, 4, 5
f8, ft7, fc3 = 6, 7, 8
fcz, fc4, ft8 = 9, 10, 11
t3, c3, cz = 12, 13, 14
c4, t4, tp7 = 15, 16, 17
cp3, cpz, cp4 = 18, 19, 20
tp8a1, t5, p3 = 21, 22, 23
pz, p4, t6a2 = 24, 25, 26
o1, oz, o2 = 27, 28, 29

# Subject boundaries (original sample indices)
startsub = np.zeros(11)
finalsub = np.zeros(11)
startsub[0] = 0
finalsub[0] = 89
startsub[1] = finalsub[0]
finalsub[1] = 468
startsub[2] = finalsub[1]
finalsub[2] = 642
startsub[3] = finalsub[2]
finalsub[3] = 757
startsub[4] = finalsub[3]
finalsub[4] = 842
startsub[5] = finalsub[4]
finalsub[5] = 1008
startsub[6] = finalsub[5]
finalsub[6] = 1110
startsub[7] = finalsub[6]
finalsub[7] = 1374
startsub[8] = finalsub[7]
finalsub[8] = 1688
startsub[9] = finalsub[8]
finalsub[9] = 1796
startsub[10] = finalsub[9]
finalsub[10] = 2022


def bipolar(xdata):
    xdatabipolar = np.zeros((2022, 32, 384))
    # vertical
    # 1
    xdatabipolar[:, 0, :] = xdata[:, fp1, :] - xdata[:, f7, :]
    xdatabipolar[:, 1, :] = xdata[:, f7, :] - xdata[:, t3, :]
    xdatabipolar[:, 2, :] = xdata[:, t3, :] - xdata[:, t5, :]
    xdatabipolar[:, 3, :] = xdata[:, t5, :] - xdata[:, o1, :]
    xdatabipolar[:, 4, :] = xdata[:, o1, :] - xdata[:, p3, :]
    xdatabipolar[:, 5, :] = xdata[:, p3, :] - xdata[:, c3, :]
    xdatabipolar[:, 6, :] = xdata[:, c3, :] - xdata[:, f3, :]
    xdatabipolar[:, 7, :] = xdata[:, f3, :] - xdata[:, fp1, :]
    # 2
    xdatabipolar[:, 8, :] = xdata[:, pz, :] - xdata[:, cz, :]
    xdatabipolar[:, 9, :] = xdata[:, cz, :] - xdata[:, fz, :]
    # 3
    xdatabipolar[:, 10, :] = xdata[:, fp2, :] - xdata[:, f8, :]
    xdatabipolar[:, 11, :] = xdata[:, f8, :] - xdata[:, t4, :]
    xdatabipolar[:, 12, :] = xdata[:, t4, :] - xdata[:, tp8a1, :]
    xdatabipolar[:, 13, :] = xdata[:, tp8a1, :] - xdata[:, o2, :]
    xdatabipolar[:, 14, :] = xdata[:, o2, :] - xdata[:, p4, :]
    xdatabipolar[:, 15, :] = xdata[:, p4, :] - xdata[:, c4, :]
    xdatabipolar[:, 16, :] = xdata[:, c4, :] - xdata[:, f4, :]
    xdatabipolar[:, 17, :] = xdata[:, f4, :] - xdata[:, fp2, :]

    # horizontal
    # 1
    xdatabipolar[:, 18, :] = xdata[:, fp1, :] - xdata[:, f3, :]
    xdatabipolar[:, 19, :] = xdata[:, f3, :] - xdata[:, c3, :]
    xdatabipolar[:, 20, :] = xdata[:, c3, :] - xdata[:, p3, :]
    xdatabipolar[:, 21, :] = xdata[:, p3, :] - xdata[:, o1, :]

    # 2
    xdatabipolar[:, 22, :] = xdata[:, fp2, :] - xdata[:, f4, :]
    xdatabipolar[:, 23, :] = xdata[:, f4, :] - xdata[:, c4, :]
    xdatabipolar[:, 24, :] = xdata[:, c4, :] - xdata[:, p4, :]
    xdatabipolar[:, 25, :] = xdata[:, p4, :] - xdata[:, o2, :]

    # 3
    xdatabipolar[:, 26, :] = xdata[:, f7, :] - xdata[:, fc3, :]
    xdatabipolar[:, 27, :] = xdata[:, fc3, :] - xdata[:, t3, :]
    xdatabipolar[:, 28, :] = xdata[:, t3, :] - xdata[:, cp3, :]
    xdatabipolar[:, 29, :] = xdata[:, cp3, :] - xdata[:, t5, :]

    # 4
    xdatabipolar[:, 30, :] = xdata[:, f8, :] - xdata[:, fc4, :]
    xdatabipolar[:, 31, :] = xdata[:, fc4, :] - xdata[:, t4, :]

    return xdatabipolar


def overlap_data(xdata, label):
    newdata = np.zeros((6064, xdata.shape[1], xdata.shape[2]))
    newlabel = np.zeros((6064,))
    leni1 = 0
    for iindex in range(0, 11):
        s = startsub[iindex]
        f = finalsub[iindex]
        leni2 = leni1 + int(f - s)
        for i in range(leni1, leni2 - 1):
            newdata[(3 * i), :, :] = xdata[i, :, :]

            newdata[(3 * i) + 1, :, :256] = xdata[i, :, 128:]
            newdata[(3 * i) + 1, :, 256:] = xdata[i + 1, :, :128]

            newdata[(3 * i) + 2, :, :128] = xdata[i, :, 256:]
            newdata[(3 * i) + 2, :, 128:] = xdata[i + 1, :, :256]

        for i in range(leni1, leni2 - 1):
            newlabel[(3 * i)] = label[i]
            newlabel[(3 * i) + 1] = label[i]
            newlabel[(3 * i) + 2] = label[i + 1]

        leni1 = leni2
    return newdata, newlabel


total 173M
-rw-r--r-- 1 root root 173M Jan  3 19:51 dataset.mat
---------- 1 root root  34K Jan  3 19:51 __notebook__.ipynb

✓ Dataset copied


---
Cell 1 — Preprocessing
---
---

In [2]:
mat_path = "/kaggle/input/dataset-drowsiness/dataset (1).mat"
mat_contents = sio.loadmat(mat_path, squeeze_me=True, struct_as_record=False)

# کلیدهای فایل (برای چک)
keys = [k for k in mat_contents.keys() if not k.startswith("__")]
print("کلیدهای داخل فایل:", keys)

# داده و برچسب مطابق فایل تو
data = np.array(mat_contents["EEGsample"])          # (2022, 30, 384)
labels = np.array(mat_contents["substate"]).reshape(-1)  # (2022,)

print("data shape:", data.shape)
print("labels shape:", labels.shape)

# Create subject index vector (1..11) aligned to your startsub/finalsub
subIdx = np.zeros((2022,), dtype=int)
for si in range(11):
    s = int(startsub[si]); f = int(finalsub[si])
    subIdx[s:f] = si + 1

processed_data = data.copy()
processed_labels = labels.copy()

if ENABLE_BIPOLAR == 1:
    processed_data = bipolar(processed_data)

if ENABLE_OVERLAP_AUGMENT == 1:
    processed_data, processed_labels = overlap_data(processed_data, processed_labels)

    # rebuild subject groups for augmented samples
    base_idx = np.minimum(np.arange(processed_data.shape[0]) // 3, 2021)
    subIdx = subIdx[base_idx]

if ENABLE_WITHIN_SUBJECT_STANDARDIZE == 1:
    # standardize per subject across time for each channel
    x = processed_data
    for subj in range(1, 12):
        mask = (subIdx == subj)
        if not np.any(mask):
            continue
        xs = x[mask]  # (n, ch, t)
        mean = xs.mean(axis=(0, 2), keepdims=True)
        std = xs.std(axis=(0, 2), keepdims=True) + 1e-12
        x[mask] = (xs - mean) / std
    processed_data = x

print("processed_data:", processed_data.shape, "processed_labels:", processed_labels.shape, "subIdx:", subIdx.shape)


کلیدهای داخل فایل: ['EEGsample', 'subindex', 'substate']
data shape: (2022, 30, 384)
labels shape: (2022,)
processed_data: (6064, 32, 384) processed_labels: (6064,) subIdx: (6064,)


---
Cell 2 — Feature Extraction
---
---

In [3]:
def _bandpower_from_psd(freqs, psd, fmin, fmax):
    mask = (freqs >= fmin) & (freqs < fmax)
    if not np.any(mask):
        return 0.0
    return np.trapz(psd[mask], freqs[mask])

def _hjorth_params(x):
    x = np.asarray(x)
    dx = np.diff(x)
    ddx = np.diff(dx)
    var_x = np.var(x) + 1e-12
    var_dx = np.var(dx) + 1e-12
    var_ddx = np.var(ddx) + 1e-12
    activity = var_x
    mobility = np.sqrt(var_dx / var_x)
    complexity = np.sqrt(var_ddx / var_dx) / (mobility + 1e-12)
    return activity, mobility, complexity

def extract_eeg_features(X, fs, bands):
    # X: (n_samples, n_channels, n_times)
    n, c, t = X.shape
    feats = []
    freqs = np.fft.rfftfreq(t, d=1.0/fs)

    for i in range(n):
        sample_feats = []
        for ch in range(c):
            x = X[i, ch, :]
            fft = np.fft.rfft(x)
            psd = (np.abs(fft) ** 2) / (t + 1e-12)

            # Relative band powers
            total = _bandpower_from_psd(freqs, psd, 0.5, 45.0) + 1e-12
            rel = []
            for _, (fmin, fmax) in bands.items():
                bp = _bandpower_from_psd(freqs, psd, fmin, fmax)
                rel.append(bp / total)

            # Ratios
            theta_alpha = (rel[list(bands.keys()).index("theta")] + 1e-12) / (rel[list(bands.keys()).index("alpha")] + 1e-12)
            theta_beta  = (rel[list(bands.keys()).index("theta")] + 1e-12) / (rel[list(bands.keys()).index("beta")] + 1e-12)

            # Hjorth
            act, mob, comp = _hjorth_params(x)

            # Spectral entropy
            p = psd / (np.sum(psd) + 1e-12)
            spec_entropy = -np.sum(p * np.log(p + 1e-12)) / (np.log(len(p)) + 1e-12)

            sample_feats.extend(rel)
            sample_feats.extend([theta_alpha, theta_beta])
            sample_feats.extend([act, mob, comp])
            sample_feats.append(spec_entropy)

        feats.append(sample_feats)

    return np.asarray(feats, dtype=np.float64)

feature_X = None
feature_y = None
feature_groups = None


# اگر مسیر ویژگی فعال باشد ولی استخراج ویژگی خاموش باشد، برای جلوگیری از توقف برنامه آن را روشن می‌کنیم
if PIPELINE_MODE == 1 and ENABLE_FEATURE_EXTRACTION == 0:
    print("[استخراج ویژگی] چون مسیر ویژگی فعال است، استخراج ویژگی به صورت خودکار روشن شد")
    ENABLE_FEATURE_EXTRACTION = 1

if ENABLE_FEATURE_EXTRACTION == 1:
    print("[Feature Extraction] Computing EEG features...")
    feature_X = extract_eeg_features(processed_data, FS, BANDS)
    feature_y = processed_labels.astype(int).copy()
    feature_groups = subIdx.astype(int).copy()
    print(f"[Feature Extraction] feature_X shape: {feature_X.shape}")
else:
    print("[Feature Extraction] Disabled")


[استخراج ویژگی] چون مسیر ویژگی فعال است، استخراج ویژگی به صورت خودکار روشن شد
[Feature Extraction] Computing EEG features...


/tmp/ipykernel_23/367013577.py:5: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(psd[mask], freqs[mask])


[Feature Extraction] feature_X shape: (6064, 352)


---
Cell 3 — Feature Selection
---
---

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif

def fit_transform_selector(X_train, y_train, X_test, method="f_classif", k=200):
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_test_s = scaler.transform(X_test)

    if k is None or k <= 0 or k >= X_train_s.shape[1]:
        return X_train_s, X_test_s, scaler, None

    if method == "mutual_info":
        selector = SelectKBest(score_func=mutual_info_classif, k=k)
    else:
        selector = SelectKBest(score_func=f_classif, k=k)

    X_train_k = selector.fit_transform(X_train_s, y_train)
    X_test_k = selector.transform(X_test_s)
    return X_train_k, X_test_k, scaler, selector


---
Cell 4 — Model Training
---
---

In [5]:
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

# Storage for results
all_results_raw = np.zeros((N_RUNS, 11), dtype=float)
all_results_smooth = np.full((N_RUNS, 11), np.nan, dtype=float)

# Store detailed rows for Excel export
fold_rows = []

# Optional: store predictions to inspect later (per run, per subject)
stored_predictions = {}

# Define CNN model only if used
class InterpretableCNN(nn.Module):
    def __init__(self, n_channels):
        super().__init__()
        self.temporal = nn.Conv2d(1, 8, kernel_size=(1, 64), stride=(1, 1), padding=(0, 32), bias=False)
        self.spatial = nn.Conv2d(8, 16, kernel_size=(n_channels, 1), bias=False)
        self.bn = nn.BatchNorm2d(16)
        self.pool = nn.AvgPool2d(kernel_size=(1, 8), stride=(1, 8))
        self.drop = nn.Dropout(0.5)
        self.classifier = nn.Linear(16 * (processed_data.shape[2] // 8), 2)

    def forward(self, x):
        x = self.temporal(x)
        x = self.spatial(x)
        x = self.bn(x)
        x = x ** 2
        x = self.pool(x)
        x = torch.log(torch.clamp(x, min=1e-6))
        x = self.drop(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return nn.functional.log_softmax(x, dim=1)

# LOSO for each run (different random_state for LR, CNN shuffle)
for run in range(N_RUNS):
    print(f"\nRun {run+1}/{N_RUNS}")
    for test_subj in range(1, 12):
        test_idx = np.where(subIdx == test_subj)[0]
        train_mask = (subIdx != test_subj)

        if PIPELINE_MODE == 0:
            # CNN on raw time-series
            X_train = processed_data[train_mask]
            y_train = processed_labels[train_mask].astype(int)
            X_test = processed_data[test_idx]
            y_test = processed_labels[test_idx].astype(int)

            # Torch expects (N, 1, C, T)
            X_train_t = torch.tensor(X_train).unsqueeze(1).double()
            y_train_t = torch.tensor(y_train).long()
            X_test_t = torch.tensor(X_test).unsqueeze(1).double()
            y_test_t = torch.tensor(y_test).long()

            train_dataset = torch.utils.data.TensorDataset(X_train_t, y_train_t)
            train_loader = torch.utils.data.DataLoader(
                train_dataset, batch_size=BATCH_SIZE, shuffle=True
            )

            n_channels = X_train.shape[1]
            model = InterpretableCNN(n_channels=n_channels).double().to(device)
            optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
            criterion = nn.NLLLoss()

            model.train()
            for epoch in range(N_EPOCHS):
                for batch_x, batch_y in train_loader:
                    batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                    optimizer.zero_grad()
                    output = model(batch_x)
                    loss = criterion(output, batch_y)
                    loss.backward()
                    optimizer.step()

            model.eval()
            with torch.no_grad():
                logits = model(X_test_t.to(device))
                pred_raw = torch.argmax(logits, dim=1).cpu().numpy().astype(int)
                y_true = y_test.astype(int)
                acc_raw = accuracy_score(y_true, pred_raw)

            pred_smooth = None
            acc_smooth = np.nan
            if ENABLE_POSTPROCESSING == 1:
                pred_smooth = smooth_predictions(pred_raw, window=SMOOTH_WINDOW).astype(int)
                acc_smooth = accuracy_score(y_true, pred_smooth)

        else:
            # Feature-based ML
            if feature_X is None or feature_y is None or feature_groups is None:
                raise RuntimeError("ویژگی‌ها ساخته نشده‌اند. مطمئن شو که دادهٔ ورودی درست خوانده شده و بخش استخراج ویژگی اجرا شده است.")

            X_train = feature_X[train_mask]
            y_train = feature_y[train_mask].astype(int)

            X_test = feature_X[test_idx]
            y_test = feature_y[test_idx].astype(int)
            y_true = y_test

            if ENABLE_FEATURE_SELECTION == 1:
                X_train, X_test, scaler, selector = fit_transform_selector(
                    X_train, y_train, X_test, method=FEATURE_SELECTION_METHOD, k=FEATURE_K
                )
            else:
                X_train, X_test, scaler, selector = fit_transform_selector(
                    X_train, y_train, X_test, method=FEATURE_SELECTION_METHOD, k=0
                )

            clf = LogisticRegression(
                max_iter=5000,
                class_weight="balanced",
                solver="lbfgs",
                random_state=run,
                n_jobs=None
            )
            clf.fit(X_train, y_train)

            pred_raw = clf.predict(X_test).astype(int)
            acc_raw = accuracy_score(y_true, pred_raw)

            pred_smooth = None
            acc_smooth = np.nan
            if ENABLE_POSTPROCESSING == 1:
                pred_smooth = smooth_predictions(pred_raw, window=SMOOTH_WINDOW).astype(int)
                acc_smooth = accuracy_score(y_true, pred_smooth)

        # confusion matrices for reporting
        cm_raw = confusion_matrix(y_true, pred_raw, labels=[0, 1]).ravel()
        if cm_raw.size == 4:
            tn_raw, fp_raw, fn_raw, tp_raw = cm_raw
        else:
            tn_raw = fp_raw = fn_raw = tp_raw = 0

        if ENABLE_POSTPROCESSING == 1 and pred_smooth is not None:
            cm_s = confusion_matrix(y_true, pred_smooth, labels=[0, 1]).ravel()
            if cm_s.size == 4:
                tn_s, fp_s, fn_s, tp_s = cm_s
            else:
                tn_s = fp_s = fn_s = tp_s = 0
        else:
            tn_s = fp_s = fn_s = tp_s = np.nan

        fold_rows.append({
            "run": run + 1,
            "subject": test_subj,
            "n_samples": int(len(y_true)),
            "acc_raw": float(acc_raw),
            "acc_smooth": float(acc_smooth) if ENABLE_POSTPROCESSING == 1 else np.nan,
            "tn_raw": int(tn_raw), "fp_raw": int(fp_raw), "fn_raw": int(fn_raw), "tp_raw": int(tp_raw),
            "tn_smooth": float(tn_s), "fp_smooth": float(fp_s), "fn_smooth": float(fn_s), "tp_smooth": float(tp_s),
        })

        all_results_raw[run, test_subj - 1] = acc_raw
        if ENABLE_POSTPROCESSING == 1:
            all_results_smooth[run, test_subj - 1] = acc_smooth

        stored_predictions[(run + 1, test_subj)] = {
            "y_true": y_true,
            "pred_raw": pred_raw,
            "pred_smooth": pred_smooth if ENABLE_POSTPROCESSING == 1 else None,
        }

        if ENABLE_POSTPROCESSING == 1:
            print(f"  Subject {test_subj:2d}: raw={acc_raw:.4f} | smooth={acc_smooth:.4f}")
        else:
            print(f"  Subject {test_subj:2d}: {acc_raw:.4f}")

    print(f"Run mean (raw): {np.mean(all_results_raw[run]):.4f}")
    if ENABLE_POSTPROCESSING == 1:
        print(f"Run mean (smooth): {np.nanmean(all_results_smooth[run]):.4f}")


device: cuda

Run 1/11
  Subject  1: raw=0.4045 | smooth=0.3820
  Subject  2: raw=0.6394 | smooth=0.6640
  Subject  3: raw=0.8103 | smooth=0.8295
  Subject  4: raw=0.6899 | smooth=0.7217
  Subject  5: raw=0.4863 | smooth=0.4863
  Subject  6: raw=0.7731 | smooth=0.8273
  Subject  7: raw=0.6176 | smooth=0.6275
  Subject  8: raw=0.5657 | smooth=0.5657
  Subject  9: raw=0.7994 | smooth=0.8556
  Subject 10: raw=0.8056 | smooth=0.8241
  Subject 11: raw=0.5843 | smooth=0.6050
Run mean (raw): 0.6524
Run mean (smooth): 0.6717

Run 2/11
  Subject  1: raw=0.4045 | smooth=0.3820
  Subject  2: raw=0.6394 | smooth=0.6640
  Subject  3: raw=0.8103 | smooth=0.8295
  Subject  4: raw=0.6899 | smooth=0.7217
  Subject  5: raw=0.4863 | smooth=0.4863
  Subject  6: raw=0.7731 | smooth=0.8273
  Subject  7: raw=0.6176 | smooth=0.6275
  Subject  8: raw=0.5657 | smooth=0.5657
  Subject  9: raw=0.7994 | smooth=0.8556
  Subject 10: raw=0.8056 | smooth=0.8241
  Subject 11: raw=0.5843 | smooth=0.6050
Run mean (raw): 

---
Cell 5 — Post-processing
---
---

In [6]:
def smooth_predictions(pred, window=5):
    pred = np.asarray(pred).astype(int)
    if window is None or window <= 1:
        return pred
    if window % 2 == 0:
        window += 1

    pad = window // 2
    padded = np.pad(pred, (pad, pad), mode="edge")
    out = pred.copy()

    for i in range(len(pred)):
        w = padded[i:i + window]
        out[i] = 1 if np.mean(w) >= 0.5 else 0
    return out


---
Cell 6 — Evaluation
---
---

In [7]:
import pandas as pd

print("\nOverall mean (raw):", np.mean(all_results_raw))
print("Overall std  (raw):", np.std(all_results_raw))

if ENABLE_POSTPROCESSING == 1:
    print("Overall mean (post-processed):", np.nanmean(all_results_smooth))
    print(f"Overall std  (post-processed): {np.nanstd(all_results_smooth):.4f}")

# Save numpy results
np.save("results_raw.npy", all_results_raw)
if ENABLE_POSTPROCESSING == 1:
    np.save("results_smooth.npy", all_results_smooth)

# Build Excel tables
fold_df = pd.DataFrame(fold_rows)

# Per-subject summary
summary_cols = ["acc_raw"]
if ENABLE_POSTPROCESSING == 1:
    summary_cols.append("acc_smooth")

subj_summary = (
    fold_df.groupby("subject")[summary_cols]
    .agg(["mean", "std", "count"])
    .reset_index()
)

# Flatten multi-index columns for Excel readability
subj_summary.columns = ["_".join([c for c in col if c]) for col in subj_summary.columns.values]

# Overall summary
overall = {}
overall["metric"] = "raw"
overall["mean"] = float(np.mean(all_results_raw))
overall["std"] = float(np.std(all_results_raw))
overall_rows = [overall]

if ENABLE_POSTPROCESSING == 1:
    overall_rows.append({
        "metric": "smooth",
        "mean": float(np.nanmean(all_results_smooth)),
        "std": float(np.nanstd(all_results_smooth)),
    })

overall_df = pd.DataFrame(overall_rows)

excel_path = "نتایج_نهایی.xlsx"
with pd.ExcelWriter(excel_path, engine="openpyxl") as writer:
    fold_df.to_excel(writer, index=False, sheet_name="folds")
    subj_summary.to_excel(writer, index=False, sheet_name="per_subject")
    overall_df.to_excel(writer, index=False, sheet_name="overall")

print("\n✓ Results saved")
print(f"✓ Excel saved: {excel_path}")



Overall mean (raw): 0.6523640782053125
Overall std  (raw): 0.1309273962846851
Overall mean (post-processed): 0.671701176539106
Overall std  (post-processed): 0.1491

✓ Results saved
✓ Excel saved: نتایج_نهایی.xlsx
